In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
from io import StringIO

## Wikipedia scrapping with pandas

In [3]:
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [29]:
#exaple on how to get the values
#df[0].loc[5,5].split('(')[1].split(')')[0]

'Dufferin / Dovercourt Village'

#### Select the table that will be used for this assignement

In [4]:
df2= df[0]

,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park / Ontario Provincial Government,M8ANot assigned,M9AEtobicoke(Islington Avenue)


In [40]:
df2.size

180

## Parse data 

In [241]:
row_list = []
for j in (range (0,df2[0].count())):
    for i in df2.loc[j,::]:
        neighborhood = ""
        temp = i.split('(')
        #print("TEMP", temp, len(temp))
        post_code = i[0:3]
        #print("POST_CODE:",post_code)
        borough = temp[0][3::]
        #print("borough:",borough)
        if (len(temp)>1):
            temp2 = temp[1].split(')')
            #print('Temp2: ', len(temp2), temp2)
            if '/' in temp2[0]:
                neighborhood = temp2[0].split('/')
                neighborhood = ','.join(neighborhood)
                #neighborhood = temp2[0]
            elif ',' in temp2[0]:
                neighborhood = temp2[0].split(',')
                #neighborhood = temp2[0]
                neighborhood = ','.join(neighborhood)
            else:
                neighborhood = temp2[0]
        #print('Neighborhood:', neighborhood)
        row_list.append({'Postal Code': post_code,'Borough':borough,'Neighborhood':neighborhood})

    

In [45]:
results = pd.DataFrame(row_list, columns=['Postal Code','Borough','Neighborhood'])

In [46]:
results.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"


### Filter out the Borough that are not assigned

In [47]:
results = results[results['Borough'] != "Not assigned"]
results.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Queen's Park / Ontario Provincial Government,


In [51]:
results.groupby(['Borough']).count()

,Postal Code,Neighborhood
Borough,,
Central Toronto,9,9
Downtown Toronto,17,17
Downtown TorontoStn A PO Boxes25 The Esplanade,1,1
East Toronto,4,4
East TorontoBusiness reply mail Processing Centre969 Eastern,1,1
East York,4,4
East YorkEast Toronto,1,1
Etobicoke,11,11
EtobicokeNorthwest,1,1


### Some data needs to be corrected manually

In [52]:
#based on the group by - clean all the elements that seems wrong
results.replace("Queen's Park / Ontario Provincial Government", "Queen's Park",inplace=True)

results.replace("Queen's Park / Ontario Provincial Government", "Queen's Park",inplace=True)

results.replace("MississaugaCanada Post Gateway Processing Centre", "Mississauga",inplace=True)

results.replace("East TorontoBusiness reply mail Processing Centre969 Eastern", "East Toronto",inplace=True)

results.replace("Downtown TorontoStn A PO Boxes25 The Esplanade", "Downtown Toronto",inplace=True)

results.replace("EtobicokeNorthwest", "Etobicoke",inplace=True)

results.replace("EtobicokeNorthwest", "Etobicoke",inplace=True)

results.replace("East YorkEast Toronto", "East York",inplace=True)

results.replace("EtobicokeNorthwest", "Etobicoke",inplace=True)

In [53]:
results.shape

(103, 3)

In [55]:
results = results.reset_index().drop('index',axis=1)

In [94]:
results.loc[4,'Neighborhood'] = results.loc[4,'Borough']

In [95]:
results

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [96]:
results.to_csv("toronto_results")

In [ ]:
#results.loc[0,'Borough']

### Get latitude and longitude

In [97]:
# def geo_lat(ad):
#     try:
#         geolocator = Nominatim(user_agent="foursquare_agent")
#         location = geolocator.geocode(ad)
#         latitude = location.latitude
#         longitude = location.longitude
#     except:
#         latitude=0.0
#         longitude=0.0
#         print(ad)
#     return (latitude,longitude)

def geo_lat(ad):
    try:
        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(ad)
        latitude = location.latitude
        longitude = location.longitude
    except:
        latitude=0.0
        longitude=0.0
        print(ad)
    return latitude
def geo_lon(ad):
    try:
        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(ad)
        latitude = location.latitude
        longitude = location.longitude
    except:
        longitude = 0.0
        print(ad)
    return longitude

In [118]:
results['Longitude'] = results['Borough'].apply(lambda x: geo_lon(x))

In [119]:
results['Latitude'] = results['Borough'].apply(lambda x: geo_lat(x))

Etobicoke


### Select a subset of the data to run the analysis. Select all Borough that contains Toronto in name

In [120]:
toronto_data = results[results['Borough'].str.contains('Toronto', regex=False)].reset_index().drop('index',1)

In [121]:
toronto_data['Ne'] = toronto_data['Neighborhood'].apply(lambda x: x.split(','))

In [122]:
temp = toronto_data['Ne'].apply(pd.Series)

In [123]:
temp.tail()

,0,1,2,3,4,5,6
33,Enclave of M5E,NaN,NaN,NaN,NaN,NaN,NaN
34,St. James Town,Cabbagetown,NaN,NaN,NaN,NaN,NaN
35,First Canadian Place,Underground city,NaN,NaN,NaN,NaN,NaN
36,Church and Wellesley,NaN,NaN,NaN,NaN,NaN,NaN
37,Enclave of M4L,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
temp2 = temp.stack().reset_index().drop('level_1',1)

In [125]:
temp2.head()

,level_0,0
0,0,Regent Park
1,0,Harbourfront
2,1,Garden District
3,1,Ryerson
4,2,St. James Town


In [126]:
temp2.columns = ['prev_index','Neighborhood']

In [127]:
temp2

,prev_index,Neighborhood
0,0,Regent Park
1,0,Harbourfront
2,1,Garden District
3,1,Ryerson
4,2,St. James Town
5,3,The Beaches
6,4,Berczy Park
7,5,Central Bay Street
8,6,Christie
9,7,Richmond


In [128]:
toronto_data['prev_index'] = range(0,len(toronto_data.index))

In [129]:
temp2.head()

,prev_index,Neighborhood
0,0,Regent Park
1,0,Harbourfront
2,1,Garden District
3,1,Ryerson
4,2,St. James Town


In [130]:
toronto_data2.head()

,prev_index,Neighborhood_x,Postal Code,Borough,Neighborhood_y,Ne
0,0,Regent Park,M5A,Downtown Toronto,"Regent Park , Harbourfront","[Regent Park , Harbourfront]"
1,0,Harbourfront,M5A,Downtown Toronto,"Regent Park , Harbourfront","[Regent Park , Harbourfront]"
2,1,Garden District,M5B,Downtown Toronto,"Garden District, Ryerson","[Garden District, Ryerson]"
3,1,Ryerson,M5B,Downtown Toronto,"Garden District, Ryerson","[Garden District, Ryerson]"
4,2,St. James Town,M5C,Downtown Toronto,St. James Town,[St. James Town]


In [131]:
toronto_data2 = pd.merge(temp2, toronto_data, on='prev_index')

In [132]:
toronto_final = toronto_data2[['Postal Code','Borough','Neighborhood_x','Latitude', 'Longitude']]

In [133]:
toronto_final = toronto_final[~toronto_final['Neighborhood_x'].isin(['Enclave of M5E','Enclave of M4L'])]

### the Latitude and Longitude are coresponding to the Borough at this point. it will be corrected below

In [134]:
toronto_final

,Postal Code,Borough,Neighborhood_x,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park,43.654174,-79.380812
1,M5A,Downtown Toronto,Harbourfront,43.654174,-79.380812
2,M5B,Downtown Toronto,Garden District,43.654174,-79.380812
3,M5B,Downtown Toronto,Ryerson,43.654174,-79.380812
4,M5C,Downtown Toronto,St. James Town,43.654174,-79.380812
5,M4E,East Toronto,The Beaches,43.653963,-79.387207
6,M5E,Downtown Toronto,Berczy Park,43.654174,-79.380812
7,M5G,Downtown Toronto,Central Bay Street,43.654174,-79.380812
8,M6G,Downtown Toronto,Christie,43.654174,-79.380812
9,M5H,Downtown Toronto,Richmond,43.654174,-79.380812


#### strip down the spaces in the Neighborhood_x

In [135]:
toronto_final['Neighborhood_x']= toronto_final['Neighborhood_x'].str.strip()

In [138]:
toronto_final[toronto_final['Neighborhood_x'].str.contains('Enclave', regex=False)]

,Postal Code,Borough,Neighborhood_x,Latitude,Longitude


In [141]:
toronto_final['Longitude_Neigh'] = toronto_final.apply(lambda x : geo_lon('Toronto '+x['Neighborhood_x']), axis=1)

Toronto Commerce Court
Toronto Studio District
Toronto Railway Lands


In [142]:
toronto_final['Latitude_Neigh'] = toronto_final.apply(lambda x : geo_lat('Toronto '+x['Neighborhood_x']), axis=1)
#toronto_final[['Latitude_Neigh','Longitude_Neigh']] = toronto_final.apply(lambda x : geo_lat('Toronto '+x['Neighborhood_x']), axis=1)

Toronto Studio District
Toronto Railway Lands


### Based on the results from previous neigh lat long we will correct manually several items
- Toronto Commerce Court
- Toronto Studio District
- Toronto Railway Lands

The data corrected manually will vary depending on several factors like (response of the geolocator)

In [139]:
# for i in range(0,toronto_final['Neighborhood_x'].count()):
#     b = toronto_final.loc[i,'Neighborhood_x']
#     print(b)
#     geolocator = Nominatim(user_agent="foursquare_agent")
#     location = geolocator.geocode(b)
#     latitude = location.latitude
#     longitude = location.longitude
#     print(latitude,longitude)

In [144]:
b = "Toronto Commerce Court" 
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(b)
latitude = location.latitude
longitude = location.longitude
print(latitude,longitude)

43.64809515 -79.37902543099575


In [145]:
toronto_final[toronto_final['Neighborhood_x'].str.contains('Commerce Court')]

,Postal Code,Borough,Neighborhood_x,Latitude,Longitude,Longitude_Neigh,Latitude_Neigh
28,M5L,Downtown Toronto,Commerce Court,43.654174,-79.380812,0.0,43.648095


In [146]:
toronto_final.loc[28,['Latitude_Neigh','Longitude_Neigh']]  = [latitude,longitude]

In [156]:
b = "Railway Lands Toronto " 
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(b)
latitude = location.latitude
longitude = location.longitude
print(latitude,longitude)

49.2662356 -123.2370598


In [157]:
toronto_final[toronto_final['Neighborhood_x'].str.contains('Railway Lands')]

,Postal Code,Borough,Neighborhood_x,Latitude,Longitude,Longitude_Neigh,Latitude_Neigh
60,M5V,Downtown Toronto,Railway Lands,43.654174,-79.380812,0.0,0.0


In [158]:
toronto_final.loc[60,['Latitude_Neigh','Longitude_Neigh']]  = [latitude,longitude]

In [159]:
b = "Studio District Toronto " 
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(b)
latitude = location.latitude
longitude = location.longitude
print(latitude,longitude)

43.64958515 -79.39068322559326


In [160]:
toronto_final[toronto_final['Neighborhood_x'].str.contains('Studio District')]

,Postal Code,Borough,Neighborhood_x,Latitude,Longitude,Longitude_Neigh,Latitude_Neigh
30,M4M,East Toronto,Studio District,43.653963,-79.387207,-79.390683,43.649585


In [161]:
toronto_final.loc[30,['Latitude_Neigh','Longitude_Neigh']]  = [latitude,longitude]

In [172]:
toronto_final

,Postal Code,Borough,Neighborhood_x,Latitude,Longitude,Longitude_Neigh,Latitude_Neigh
0,M5A,Downtown Toronto,Regent Park,43.654174,-79.380812,-79.360457,43.660706
1,M5A,Downtown Toronto,Harbourfront,43.654174,-79.380812,-79.380150,43.640080
2,M5B,Downtown Toronto,Garden District,43.654174,-79.380812,-79.377114,43.656500
3,M5B,Downtown Toronto,Ryerson,43.654174,-79.380812,-79.378993,43.658469
4,M5C,Downtown Toronto,St. James Town,43.654174,-79.380812,-79.372704,43.669403
5,M4E,East Toronto,The Beaches,43.653963,-79.387207,-79.296712,43.671024
6,M5E,Downtown Toronto,Berczy Park,43.654174,-79.380812,-79.375396,43.647984
7,M5G,Downtown Toronto,Central Bay Street,43.654174,-79.380812,-79.385802,43.660708
8,M6G,Downtown Toronto,Christie,43.654174,-79.380812,-79.418405,43.664111
9,M5H,Downtown Toronto,Richmond,43.654174,-79.380812,-79.391373,43.648587


In [163]:
toronto_final.to_csv('Toronto_final.csv')

### visualize the data 

In [164]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_final['Latitude_Neigh'], toronto_final['Longitude_Neigh'], toronto_final['Borough'], toronto_final['Neighborhood_x']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Get Forursquare data 

### Define credentials and limit

### Due to the fact that i will expose the jupyter notebook to public i will remove the client and secret and we will rely on the Toronto_venues.csv that will be attached

In [167]:
CLIENT_ID = 'XXX' # your Foursquare ID
CLIENT_SECRET = 'XXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GVMIFYRIMJR425HJGDVEOXGJARHQG4BRTEOJW0ITKH45T1CH
CLIENT_SECRET:HDRO5LDFFIXOL5FOW01DO3TBJIUYKFEF301LDRNEX2H4RXMO


In [168]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [169]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [173]:
# toronto_venues = getNearbyVenues(names=toronto_final['Neighborhood_x'],
#                                    latitudes=toronto_final['Latitude_Neigh'],
#                                    longitudes=toronto_final['Longitude_Neigh']
#                                   )

In [174]:
#toronto_venues.to_csv("toronto_venues.csv")

### Read data from previosly populated csv 

In [175]:
toronto_venues = pd.read_csv("toronto_venues.csv")

In [176]:
toronto_venues.drop(['Unnamed: 0'],axis=1,inplace=True)

In [177]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park,43.660706,-79.360457,Regent Park Aquatic Centre,43.660600,-79.361392,Pool
1,Regent Park,43.660706,-79.360457,Daniels Spectrum,43.660137,-79.361808,Performing Arts Venue
2,Regent Park,43.660706,-79.360457,Sumach Espresso,43.658135,-79.359515,Coffee Shop
3,Regent Park,43.660706,-79.360457,Thai To Go,43.663418,-79.360710,Thai Restaurant
4,Regent Park,43.660706,-79.360457,Paintbox Bistro,43.660050,-79.362855,Restaurant


### Define the data that will be used in the kmeans 

In [183]:
toronto_cluster = pd.get_dummies(toronto_venues['Venue Category'])

In [184]:
toronto_cluster['Neighborhood'] = toronto_venues['Neighborhood']

In [185]:
toronto_cluster2 = toronto_cluster.groupby(['Neighborhood']).mean().reset_index()

In [186]:
toronto_cluster2.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Cafeteria,Café,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Storage Facility,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,The

In [44]:
#toronto_venues[toronto_venues['Neighborhood'] == "Adelaide"].groupby(['Venue Category']).count()

### Split data into clusters

In [187]:
kclusters = 5

toronto_grouped_clustering = toronto_cluster2.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 4, 0, 0, 0, 4, 4, 0], dtype=int32)

In [188]:
toronto_grouped_clustering.shape

(70, 300)

In [236]:
#results['Borough'].apply(lambda x: Nominatim(user_agent="foursquare_agent").geocode(x).latitude)
#results['Borough'].apply(lambda x: Nominatim(user_agent="foursquare_agent").geocode(x).longitude)

### Identify the top 10 categories in each area

In [192]:
#type(np.sort(toronto_cluster2.iloc[::,1::])[:, ::-1])

In [199]:
#example of usage
toronto_cluster2.iloc[::,1::].iloc[0,::].to_frame().sort_values(by=0, ascending=False)[0:5]


,0
Coffee Shop,0.09
Restaurant,0.06
Café,0.04
Hotel,0.04
Italian Restaurant,0.04


In [198]:
#example of usage
list(toronto_cluster2.iloc[::,1::].iloc[0,::].to_frame().sort_values(by=0, ascending=False)[0:3].T.columns)

['Coffee Shop', 'Restaurant', 'Café']

In [200]:
top10 = []
for i in range(0,toronto_cluster2.shape[0]):
    top10.append(list(toronto_cluster2.iloc[::,1::].iloc[i,::].to_frame().sort_values(by=i, ascending=False)[0:10].T.columns))


In [ ]:
#### Define a temporary df for storing the data on top categories

In [201]:
tt = toronto_cluster2.Neighborhood.to_frame()

In [203]:
tt['Top10'] = top10

In [204]:
tt.head()

,Neighborhood,Top10
0,Adelaide,"[Coffee Shop, Restaurant, Café, Hotel, Italian..."
1,Bathurst Quay,"[Coffee Shop, Café, Park, Harbor / Marina, Dan..."
2,Berczy Park,"[Coffee Shop, Café, Restaurant, Hotel, Bakery,..."
3,Brockton,"[Bar, Vietnamese Restaurant, Park, Grocery Sto..."
4,CN Tower,"[Coffee Shop, Hotel, Pizza Place, Italian Rest..."


In [205]:
tt[["1st","2nd","3rd","4th","5th","6th","7th","8th","9th","10th"]] = tt['Top10'].apply(pd.Series)

In [206]:
tt.head()

,Neighborhood,Top10,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
0,Adelaide,"[Coffee Shop, Restaurant, Café, Hotel, Italian...",Coffee Shop,Restaurant,Café,Hotel,Italian Restaurant,Seafood Restaurant,Gastropub,American Restaurant,Cosmetics Shop,Gym
1,Bathurst Quay,"[Coffee Shop, Café, Park, Harbor / Marina, Dan...",Coffee Shop,Café,Park,Harbor / Marina,Dance Studio,Tunnel,Sculpture Garden,Caribbean Restaurant,Bank,Gym
2,Berczy Park,"[Coffee Shop, Café, Restaurant, Hotel, Bakery,...",Coffee Shop,Café,Restaurant,Hotel,Bakery,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Beer Bar,Gym
3,Brockton,"[Bar, Vietnamese Restaurant, Park, Grocery Sto...",Bar,Vietnamese Restaurant,Park,Grocery Store,French Restaurant,Korean Restaurant,Café,Gastropub,Organic Grocery,Dive Bar
4,CN Tower,"[Coffee Shop, Hotel, Pizza Place, Italian Rest...",Coffee Shop,Hotel,Pizza Place,Italian Restaurant,Bar,Scenic Lookout,Baseball Stadium,Restaurant,Gym,Bistro


In [207]:
tt.to_csv('tt.csv')

In [209]:
### Test is the df that will contain the toronto clustered data 

In [210]:
test = toronto_cluster2.Neighborhood.to_frame()

In [211]:
test['Cluster'] = kmeans.labels_

In [213]:
test.head()

,Neighborhood,Cluster
0,Adelaide,0
1,Bathurst Quay,0
2,Berczy Park,0
3,Brockton,4
4,CN Tower,0


In [ ]:
### merge latitude and longitude from toronto_venues

In [214]:
temp = toronto_venues.groupby(['Neighborhood']).mean()[['Neighborhood Latitude','Neighborhood Longitude']]

In [216]:
test = pd.merge(test,temp, on='Neighborhood')

In [218]:
test.head()

,Neighborhood,Cluster,Neighborhood Latitude,Neighborhood Longitude
0,Adelaide,0,43.650486,-79.379498
1,Bathurst Quay,0,43.635790,-79.398329
2,Berczy Park,0,43.647984,-79.375396
3,Brockton,4,43.650917,-79.440022
4,CN Tower,0,43.642564,-79.387087


In [ ]:
##### come back here

In [65]:
toronto_cluster2.Neighborhood.drop(['Cluster'],inplace= True)

In [238]:
test.to_csv('test.csv')

In [76]:
test.groupby(['Cluster']).count()

,Neighborhood
Cluster,
0,36
1,1
2,4
3,2
4,27


In [ ]:
### Create the final df by merging tt and test

In [221]:
final = pd.merge(test,tt,on='Neighborhood')

In [224]:
final.drop(['Top10'],axis=1,inplace=True)

In [225]:
final.head()

,Neighborhood,Cluster,Neighborhood Latitude,Neighborhood Longitude,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
0,Adelaide,0,43.650486,-79.379498,Coffee Shop,Restaurant,Café,Hotel,Italian Restaurant,Seafood Restaurant,Gastropub,American Restaurant,Cosmetics Shop,Gym
1,Bathurst Quay,0,43.635790,-79.398329,Coffee Shop,Café,Park,Harbor / Marina,Dance Studio,Tunnel,Sculpture Garden,Caribbean Restaurant,Bank,Gym
2,Berczy Park,0,43.647984,-79.375396,Coffee Shop,Café,Restaurant,Hotel,Bakery,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Beer Bar,Gym
3,Brockton,4,43.650917,-79.440022,Bar,Vietnamese Restaurant,Park,Grocery Store,French Restaurant,Korean Restaurant,Café,Gastropub,Organic Grocery,Dive Bar
4,CN Tower,0,43.642564,-79.387087,Coffee Shop,Hotel,Pizza Place,Italian Restaurant,Bar,Scenic Lookout,Baseball Stadium,Restaurant,Gym,Bistro


In [226]:
final.to_csv('final.csv')

In [227]:
#Check top 3 categories of each cluster

In [231]:
final[final['Cluster']==0][['1st','2nd','3rd']].head()

,1st,2nd,3rd
0,Coffee Shop,Restaurant,Café
1,Coffee Shop,Café,Park
2,Coffee Shop,Café,Restaurant
4,Coffee Shop,Hotel,Pizza Place
5,Restaurant,Café,Pizza Place


In [232]:
final[final['Cluster']==1][['1st','2nd','3rd']].head()

,1st,2nd,3rd
25,Boat or Ferry,Sculpture Garden,Historic Site


In [233]:
final[final['Cluster']==2][['1st','2nd','3rd']].head()

,1st,2nd,3rd
35,Playground,Tennis Court,Gym
39,Park,Gym / Fitness Center,Light Rail Station
46,Park,Playground,Bike Trail
56,Park,Skating Rink,Pilates Studio


In [234]:
final[final['Cluster']==3][['1st','2nd','3rd']].head()

,1st,2nd,3rd
19,Accessories Store,Home Service,Bank
20,Accessories Store,Home Service,Bank


In [235]:
final[final['Cluster']==4][['1st','2nd','3rd']].head()

,1st,2nd,3rd
3,Bar,Vietnamese Restaurant,Park
7,Bar,Café,Dessert Shop
8,Korean Restaurant,Coffee Shop,Indian Restaurant
15,Café,Brazilian Restaurant,Bar
16,Bar,Bakery,Coffee Shop


In [236]:
final[final['Cluster']==5][['1st','2nd','3rd']].head()

,1st,2nd,3rd


### Visualization of the data by cluster

In [239]:
address = 'Toronto Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude,longitude)

43.653963 -79.387207


In [240]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(test['Neighborhood Latitude'], test['Neighborhood Longitude'], test['Neighborhood'], test['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters